In [1]:
%matplotlib inline

import xrft
import math
import time
import scipy.io
import numpy as np
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import matplotlib
from scipy import signal
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec

from gsw import f
from matplotlib.colors import LogNorm
from scipy.fftpack import fft2, fftn, fftshift

import dask

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_delta(lat):

    # compute the distance in km at this lon and lat
    delta_lon = 2 * np.pi * (6371 * np.cos(lat*np.pi/180)) / 360
    delta_lat = 1 * np.pi * 6371 / 180
    
    return delta_lon, delta_lat

In [3]:
# select a domain, 10 deg by 10 deg
topog = xr.open_dataset('/g/data/nm03/lxy581/synbath/SYNBATH.nc')
topog_sample = topog.z[240*135:240*145,240*145:240*155]

ydim, xdim = topog_sample.dims
nx = topog_sample[xdim].size
ny = topog_sample[ydim].size

lon = topog_sample.lon
lat = topog_sample.lat

yind = 240*140-1
xind = 240*150-1
print('Location for the centre point: \nLon: %.1f W;' % np.abs(topog.lon[xind].values),'\nLat: %.1f N.' % topog.lat[yind].values)

Location for the centre point: 
Lon: 30.0 W; 
Lat: 50.0 N.


In [4]:
lat = 50
delta_lon, delta_lat = get_delta(lat)

# topog_variance = np.nanmean(topog_sample**2)
topog_variance = np.var(topog_sample)

A_loc = 10 * delta_lon * 1e+3 * 10 * delta_lat * 1e+3 
print('Variance for the entire domain before windowing: % 3.2e' % (topog_variance.values * A_loc))

Variance for the entire domain before windowing:  2.83e+17


In [5]:
var_topog_tile = ['topog_tile_bl','topog_tile_br',
                  'topog_tile_tl','topog_tile_tr']

# bottom left 
topog_tile_bl = topog_sample[0:int(ny/2),0:int(nx/2)]
print('mean: %.2f'%np.nanmean(topog_tile_bl))

# bottom right
topog_tile_br = topog_sample[0:int(ny/2),int(nx/2):]
print('mean: %.2f'%np.nanmean(topog_tile_br))

# top left 
topog_tile_tl = topog_sample[int(ny/2):,0:int(nx/2)]
print('mean: %.2f'%np.nanmean(topog_tile_tl))

# top right
topog_tile_tr = topog_sample[int(ny/2):,int(nx/2):]
print('mean: %.2f'%np.nanmean(topog_tile_tr))

mean: -3850.94
mean: -2966.02
mean: -3124.73
mean: -3399.51


In [6]:
A_loc_tile = 5 * delta_lon * 1e+3 * 5 * delta_lat * 1e+3
total_var = 0

for var_tile in var_topog_tile:
    total_var = total_var + np.var(globals()[var_tile]).values * A_loc_tile
    print('Variance before windowing: %3.2e '% (np.var(globals()[var_tile]).values * A_loc_tile))

print(total_var)

Variance before windowing: 3.69e+16 
Variance before windowing: 5.50e+16 
Variance before windowing: 6.87e+16 
Variance before windowing: 3.52e+16 
1.957843799427504e+17


#### Reconstruct the sampled topography from demeaned tiles

In [8]:
topog_tile_bl -= np.nanmean(topog_tile_bl)
topog_tile_br -= np.nanmean(topog_tile_br)
topog_tile_tl -= np.nanmean(topog_tile_tl)
topog_tile_tr -= np.nanmean(topog_tile_tr)

In [9]:
topog_sample_re = np.zeros((ny,nx))

topog_sample_re[0:int(ny/2),0:int(nx/2)] = topog_tile_bl
topog_sample_re[0:int(ny/2),int(nx/2):] = topog_tile_br 
topog_sample_re[int(ny/2):,0:int(nx/2)] = topog_tile_tl 
topog_sample_re[int(ny/2):,int(nx/2):] = topog_tile_tr 

In [10]:
print(np.nanmean(topog_sample_re))

3.93324276432395e-05


In [12]:
topog_var_re = np.var(topog_sample_re)

A_loc = 10 * delta_lon * 1e+3 * 10 * delta_lat * 1e+3 
print('Variance for the entire domain before windowing: % 3.2e' % (topog_var_re * A_loc))

Variance for the entire domain before windowing:  1.96e+17
